In [15]:
#######################################################################################################################
# Model implemented off of https://github.com/adeshpande3/March-Madness-2017/blob/master/March%20Madness%202017.ipynb #
#######################################################################################################################
import sklearn
import pandas as pd
import numpy as np
import collections
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from sklearn.cross_validation import cross_val_score
from keras.utils import np_utils
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.ensemble import GradientBoostingRegressor
import math
import csv
%matplotlib inline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
import urllib
from sklearn.svm import LinearSVC
from utils import *
from sklearn import metrics
from sklearn.metrics import make_scorer
from Features import createSeasonDict




In [16]:
#reading input
data_dir = '../../input/'
reg_season_compact_pd = pd.read_csv(data_dir + 'RegularSeasonCompactResults.csv')
reg_season_detailed_pd = pd.read_csv(data_dir + 'RegularSeasonDetailedResults.csv')
seasons_pd = pd.read_csv(data_dir + 'Seasons.csv')
teams_pd = pd.read_csv(data_dir + 'Teams.csv')
teamList = teams_pd['TeamName'].tolist()
tourney_compact_pd = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
tourney_detailed_pd = pd.read_csv(data_dir + 'NCAATourneyDetailedResults.csv')
tourney_seeds_pd = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
conference_pd = pd.read_csv(data_dir + 'Conference.csv')
tourney_results_pd = pd.read_csv(data_dir + 'TourneyResults.csv')
NCAAChampionsList = tourney_results_pd['NCAA Champion'].tolist()

In [17]:
#loading feature matrix
#change model and test year and input data too get a different cv score and a different game result
start = 2018
end = 2019
test_year = range(start, end)
xTrain = np.load(data_dir + 'FeatureMatrix/xTrainBasic.npy')
yTrain = np.load(data_dir + 'FeatureMatrix/yTrainBasic.npy')
xTest = np.load(data_dir + 'FeatureMatrix/xTestBasic.npy')
yTest = np.load(data_dir + 'FeatureMatrix/yTestBasic.npy')
output_file = "../../submissions/Basic_2018.csv"
print(xTrain.shape)
print(xTrain)

(120575, 17)
[[  7.  15.  11. ...   5.   0.   1.]
 [-12.  -5.  10. ... -12.  -1.  -1.]
 [  9.  14.   1. ...  15.   0.   1.]
 ...
 [-11. -13.   3. ... -15.   0.   0.]
 [ -2.   6.   5. ...  19.   3.   0.]
 [  6.   0.  -9. ... -10.  -3.   0.]]


In [18]:
#models to test
#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
#model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
#model = AdaBoostClassifier(n_estimators=100)
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#neuralNetwork(10)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)]
model = linear_model.LogisticRegression()
#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
#model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
#model = AdaBoostClassifier(n_estimators=100)
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#neuralNetwork(10)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)



In [5]:
def gridSearch():
    tuned_parameters = {'penalty': ['l1','l2'], 'C': [0.001,0.003, 0.01, 0.03, 0.1,0.3, 1,3, 10,30, 100,300, 1000]}
    clf = GridSearchCV(linear_model.LogisticRegression(), tuned_parameters, cv=5,
                       scoring='neg_log_loss', n_jobs = -1)
    clf.fit(xTrain, yTrain)

    print("Best parameters set found on development set:\n")
    print(clf.best_params_)
    print("Grid scores on development set:\n")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()
    return clf.best_params_
#params = gridSearch()
model = linear_model.LogisticRegression(penalty = 'l1', C = 0.01)


In [19]:
#requires predict_proba support
def cross_validation_score():
    scores = cross_val_score(model, xTrain, yTrain, cv = 5, scoring = 'neg_log_loss')
    print "Log loss: {0} (+/- {1})".format(-scores.mean(), scores.std() * 2)
cross_validation_score()



Log loss: 0.448378599332 (+/- 0.0176120361588)


In [20]:
model.fit(xTrain, yTrain)
#print(xTest.shape)
#print(yTest.shape)
#print(xTrain.shape)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#model.fit(xTrain, yTrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
def predictGame(team_1_vector, team_2_vector, home, model):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    return model.predict_proba([diff]) #logistic regression

    #return model.predict([diff])


In [22]:
import time
t = time.time()
season_dicts = {}
for year in test_year:
    season_dicts[year] = createSeasonDict(year)
print time.time() - t

27.6909139156


In [23]:
def createPrediction():
    sample_sub_pd = pd.read_csv('../../input/SampleSubmissionStage2.csv')
    results = [[0 for x in range(2)] for x in range(len(sample_sub_pd.index))]
    for index, row in sample_sub_pd.iterrows():
        matchup_id = row['id']
        team1_id = int(matchup_id[5:9])
        team2_id = int(matchup_id[10:14])
        year = int(matchup_id[0:4])
        team1_vector = season_dicts[year][int(team1_id)]
        team2_vector = season_dicts[year][int(team2_id)]
        pred = predictGame(team1_vector, team2_vector, 0, model)
        results[index][0] = str(year) + '_'+ matchup_id[5:14]
        results[index][1] = pred[0][1]
        #results[index][1] = pred[0]
    return results

results = createPrediction()
print(results)

[['2018_1104_1112', 0.3738414299206756], ['2018_1104_1113', 0.5000021241333379], ['2018_1104_1116', 0.41039406362016], ['2018_1104_1120', 0.25795642469723395], ['2018_1104_1137', 0.8027421339931063], ['2018_1104_1138', 0.7553010520262524], ['2018_1104_1139', 0.4024987128640169], ['2018_1104_1153', 0.3171305491736425], ['2018_1104_1155', 0.39554940407967054], ['2018_1104_1158', 0.7941825110957246], ['2018_1104_1166', 0.4920298696327782], ['2018_1104_1168', 0.9260186696839756], ['2018_1104_1172', 0.7344552016099496], ['2018_1104_1181', 0.19981151370507058], ['2018_1104_1196', 0.49360553886591507], ['2018_1104_1199', 0.4493179781545124], ['2018_1104_1209', 0.8335766528726772], ['2018_1104_1211', 0.3304647030386034], ['2018_1104_1222', 0.4930479378702354], ['2018_1104_1233', 0.8860054583252439], ['2018_1104_1242', 0.22230920861996908], ['2018_1104_1243', 0.41156297515158896], ['2018_1104_1246', 0.4044084263194077], ['2018_1104_1252', 0.8979087528582783], ['2018_1104_1254', 0.97023315508682

In [24]:
def toCSV():
    firstRow = [[0 for x in range(2)] for x in range(1)]
    firstRow[0][0] = 'id'
    firstRow[0][1] = 'pred'
    with open(output_file, "wb") as f:
        writer = csv.writer(f)
        writer.writerows(firstRow)
        writer.writerows(results)
toCSV()

In [25]:
###Testing###

def predict(row, year):
    mn = min(row['WTeamID'], row['LTeamID'])
    mx = max(row['WTeamID'], row['LTeamID'])
    
    return predictions[(year, mn, mx)]

def logloss(row):
    return (row.Result * math.log(row.Prediction) + (1.0 - row.Result) * math.log(1.0 - row.Prediction))

def evaluate(year):
    ncaa_df = tourney_detailed_pd[(tourney_detailed_pd['Season'] == year) & (tourney_detailed_pd.DayNum >= 136)]
    assert(len(ncaa_df.index) == 63)
    ncaa_df['Result'] = ncaa_df.apply(lambda x : int(x.WTeamID < x.LTeamID), axis = 1)
    ncaa_df['Prediction'] = ncaa_df.apply(predict, args = (year,), axis = 1)
    ncaa_df['LogLoss'] = ncaa_df.apply(logloss, axis = 1)
    return -(ncaa_df['LogLoss'].sum() / len(ncaa_df.index))


In [26]:
predictions = {}
def test():
    import warnings
    warnings.filterwarnings("ignore")
    log_losses = []
    for row in results:
        year, sid, eid = map(int, row[0].split('_'))
        predictions[(year, sid, eid)] = row[1]
        
    for year in test_year:
        log_loss = evaluate(year)
        log_losses.append(log_loss)
        print "Log Loss of year {} is {}".format(year, log_loss)
    print "Log Loss average over years {} - {} is {}".format(test_year[0], test_year[-1], sum(log_losses) / len(log_losses))
    return sum(log_losses) / len(log_losses)

test()


AssertionError: 